In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

In [5]:
df_train = pd.read_csv("train_fktp.txt", sep=',', header=None, names=['nomor_peserta', 'nomor_keluarga', 'bobot', 'id_kunjungan_fktp', 'tgl_dtg_kunjungan_fktp', 'tgl_plg_kunjungan_fktp', 'provinsi_fktp', 'kota_fktp', 'kepemilikan_fktp', 'jenis_fktp', 'tipe_fktp', 'tingkat_pelayanan_fktp', 'jenis_poli_fktp', 'akses_layanan_fktp', 'kode_nama_diagnosis', 'kode_diagnosis', 'kode_diagnosis_2', 'nama_diagnosis', 'provinsi_rujukan', 'kota_rujukan', 'kepemilikan_faskes_rujukan', 'jenis_faskes_rujukan', 'tipe_faskes_rujukan', 'poli_faskes_rujukan', 'jenis_kunjungan_fktp', 'status_pulang'])
df_train.head()

,nomor_peserta,nomor_keluarga,bobot,id_kunjungan_fktp,tgl_dtg_kunjungan_fktp,tgl_plg_kunjungan_fktp,provinsi_fktp,kota_fktp,kepemilikan_fktp,jenis_fktp,...,kode_diagnosis_2,nama_diagnosis,provinsi_rujukan,kota_rujukan,kepemilikan_faskes_rujukan,jenis_faskes_rujukan,tipe_faskes_rujukan,poli_faskes_rujukan,jenis_kunjungan_fktp,status_pulang
0,157978748.0,50765420,20.064983,958073632,2019-08-01,2019-08-01,61,6108,3,1,...,9999,9999,98,9998,98,98,98,98,2,Sehat
1,95085112.0,26931775,2.626307,449590620P000054,2020-06-17,2020-06-17,51,5171,9,3,...,K297,"Gastritis, unspecified",98,9998,98,98,98,98,1,Belum_Sehat
2,224470578.0,227368233,1.051830,467490619P000001,2019-06-01,2019-06-01,35,3516,9,2,...,Z309,"Contraceptive management, unspecified",98,9998,98,98,98,98,1,Belum_Sehat
3,63871289.0,62313678,364.741455,254321219Y002368,2019-12-21,2019-12-21,34,3402,3,1,...,I10,Essential (primary) hypertension,98,9998,98,98,98,98,1,Belum_Sehat
4,29915626.0,113446188,1.050523,88681119P000020,2019-11-04,2019-11-04,35,3509,3,1,...,I110,Hypertensive heart disease with (congestive) h...,98,9998,98,98,98,98,1,Belum_Sehat


In [4]:
df_test = pd.read_csv("test_fktp.txt", sep=',', header=None, names=['nomor_peserta', 'nomor_keluarga', 'bobot', 'id_kunjungan_fktp', 'tgl_dtg_kunjungan_fktp', 'tgl_plg_kunjungan_fktp', 'provinsi_fktp', 'kota_fktp', 'kepemilikan_fktp', 'jenis_fktp', 'tipe_fktp', 'tingkat_pelayanan_fktp', 'jenis_poli_fktp', 'akses_layanan_fktp', 'kode_nama_diagnosis', 'kode_diagnosis', 'kode_diagnosis_2', 'nama_diagnosis', 'provinsi_rujukan', 'kota_rujukan', 'kepemilikan_faskes_rujukan', 'jenis_faskes_rujukan', 'tipe_faskes_rujukan', 'poli_faskes_rujukan', 'jenis_kunjungan_fktp'])
df_test.head()

C:\Users\User\AppData\Local\Temp\ipykernel_5860\4058400356.py:1: DtypeWarning: Columns (3,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv("test_fktp.txt", sep=',', header=None, names=['nomor_peserta', 'nomor_keluarga', 'bobot', 'id_kunjungan_fktp', 'tgl_dtg_kunjungan_fktp', 'tgl_plg_kunjungan_fktp', 'provinsi_fktp', 'kota_fktp', 'kepemilikan_fktp', 'jenis_fktp', 'tipe_fktp', 'tingkat_pelayanan_fktp', 'jenis_poli_fktp', 'akses_layanan_fktp', 'kode_nama_diagnosis', 'kode_diagnosis', 'kode_diagnosis_2', 'nama_diagnosis', 'provinsi_rujukan', 'kota_rujukan', 'kepemilikan_faskes_rujukan', 'jenis_faskes_rujukan', 'tipe_faskes_rujukan', 'poli_faskes_rujukan', 'jenis_kunjungan_fktp'])


,nomor_peserta,nomor_keluarga,bobot,id_kunjungan_fktp,tgl_dtg_kunjungan_fktp,tgl_plg_kunjungan_fktp,provinsi_fktp,kota_fktp,kepemilikan_fktp,jenis_fktp,...,kode_diagnosis,kode_diagnosis_2,nama_diagnosis,provinsi_rujukan,kota_rujukan,kepemilikan_faskes_rujukan,jenis_faskes_rujukan,tipe_faskes_rujukan,poli_faskes_rujukan,jenis_kunjungan_fktp
0,187883292.0,98048784,240.884842,954386050,2019-08-21,2019-08-21,31,3172,9,2,...,NaN,9999,9999,98,9998,98,98,98,98,2
1,123307139.0,20969999,47.378571,774246805,2019-02-12,2019-02-12,17,1705,3,1,...,NaN,9999,9999,98,9998,98,98,98,98,2
2,21416273.0,21416273,5.882927,1202010273,2020-03-27,2020-03-27,64,6402,3,1,...,NaN,9999,9999,98,9998,98,98,98,98,2
3,97630052.0,97630052,172.916031,1286640248,2020-07-30,2020-07-30,18,1872,9,2,...,NaN,9999,9999,98,9998,98,98,98,98,2
4,124531163.0,124531163,187.203140,1285179242,2020-07-28,2020-07-28,18,1809,9,2,...,NaN,9999,9999,98,9998,98,98,98,98,2


In [7]:
label_enc = LabelEncoder()

df_train.status_pulang = label_enc.fit_transform(df_train.status_pulang)
df_train.head()

,nomor_peserta,nomor_keluarga,bobot,id_kunjungan_fktp,tgl_dtg_kunjungan_fktp,tgl_plg_kunjungan_fktp,provinsi_fktp,kota_fktp,kepemilikan_fktp,jenis_fktp,...,kode_diagnosis_2,nama_diagnosis,provinsi_rujukan,kota_rujukan,kepemilikan_faskes_rujukan,jenis_faskes_rujukan,tipe_faskes_rujukan,poli_faskes_rujukan,jenis_kunjungan_fktp,status_pulang
0,157978748.0,50765420,20.064983,958073632,2019-08-01,2019-08-01,61,6108,3,1,...,9999,9999,98,9998,98,98,98,98,2,1
1,95085112.0,26931775,2.626307,449590620P000054,2020-06-17,2020-06-17,51,5171,9,3,...,K297,"Gastritis, unspecified",98,9998,98,98,98,98,1,0
2,224470578.0,227368233,1.051830,467490619P000001,2019-06-01,2019-06-01,35,3516,9,2,...,Z309,"Contraceptive management, unspecified",98,9998,98,98,98,98,1,0
3,63871289.0,62313678,364.741455,254321219Y002368,2019-12-21,2019-12-21,34,3402,3,1,...,I10,Essential (primary) hypertension,98,9998,98,98,98,98,1,0
4,29915626.0,113446188,1.050523,88681119P000020,2019-11-04,2019-11-04,35,3509,3,1,...,I110,Hypertensive heart disease with (congestive) h...,98,9998,98,98,98,98,1,0


In [8]:
x_train = df_train.iloc[:, :-1]
y_train = df_train.iloc[:, -1]

x_train

,nomor_peserta,nomor_keluarga,bobot,id_kunjungan_fktp,tgl_dtg_kunjungan_fktp,tgl_plg_kunjungan_fktp,provinsi_fktp,kota_fktp,kepemilikan_fktp,jenis_fktp,...,kode_diagnosis,kode_diagnosis_2,nama_diagnosis,provinsi_rujukan,kota_rujukan,kepemilikan_faskes_rujukan,jenis_faskes_rujukan,tipe_faskes_rujukan,poli_faskes_rujukan,jenis_kunjungan_fktp
0,157978748.0,50765420,20.064983,958073632,2019-08-01,2019-08-01,61,6108,3,1,...,NaN,9999,9999,98,9998,98,98,98,98,2
1,95085112.0,26931775,2.626307,449590620P000054,2020-06-17,2020-06-17,51,5171,9,3,...,K29,K297,"Gastritis, unspecified",98,9998,98,98,98,98,1
2,224470578.0,227368233,1.051830,467490619P000001,2019-06-01,2019-06-01,35,3516,9,2,...,Z30,Z309,"Contraceptive management, unspecified",98,9998,98,98,98,98,1
3,63871289.0,62313678,364.741455,254321219Y002368,2019-12-21,2019-12-21,34,3402,3,1,...,I10,I10,Essential (primary) hypertension,98,9998,98,98,98,98,1
4,29915626.0,113446188,1.050523,88681119P000020,2019-11-04,2019-11-04,35,3509,3,1,...,I11,I110,Hypertensive heart disease with (congestive) h...,98,9998,98,98,98,98,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4056893,212666736.0,91270403,14.246229,1285820875,2020-07-29,2020-07-29,35,3505,9,3,...,NaN,9999,9999,98,9998,98,98,98,98,2
4056894,30859241.0,30859241,3.886934,315641120P000514,2020-11-14,2020-11-14,35,3514,9,2,...,H26,H269,"Cataract, unspecified",35,3514,9,2,14,99,1
4056895,8859416.0,8859416,6.146070,962412926,2019-08-16,2019-08-16,11,1171,4,2,...,NaN,9999,9999,98,9998,98,98,98,98,2
4056896,399491834.0,399491834,26.548826,273170420P000003,2020-04-01,2020-04-01,74,7402,3,1,...,K30,K30,Dyspepsia,98,9998,98,98,98,98,1


In [10]:
x_test = df_test.iloc[:, :]

x_test

,nomor_peserta,nomor_keluarga,bobot,id_kunjungan_fktp,tgl_dtg_kunjungan_fktp,tgl_plg_kunjungan_fktp,provinsi_fktp,kota_fktp,kepemilikan_fktp,jenis_fktp,...,kode_diagnosis,kode_diagnosis_2,nama_diagnosis,provinsi_rujukan,kota_rujukan,kepemilikan_faskes_rujukan,jenis_faskes_rujukan,tipe_faskes_rujukan,poli_faskes_rujukan,jenis_kunjungan_fktp
0,187883292.0,98048784,240.884842,954386050,2019-08-21,2019-08-21,31,3172,9,2,...,NaN,9999,9999,98,9998,98,98,98,98,2
1,123307139.0,20969999,47.378571,774246805,2019-02-12,2019-02-12,17,1705,3,1,...,NaN,9999,9999,98,9998,98,98,98,98,2
2,21416273.0,21416273,5.882927,1202010273,2020-03-27,2020-03-27,64,6402,3,1,...,NaN,9999,9999,98,9998,98,98,98,98,2
3,97630052.0,97630052,172.916031,1286640248,2020-07-30,2020-07-30,18,1872,9,2,...,NaN,9999,9999,98,9998,98,98,98,98,2
4,124531163.0,124531163,187.203140,1285179242,2020-07-28,2020-07-28,18,1809,9,2,...,NaN,9999,9999,98,9998,98,98,98,98,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014220,51801010.0,51801010,7.563763,248870919P000406,2019-09-08,2019-09-08,36,3601,9,2,...,Z99,Z992,Dependence on renal dialysis,36,3602,3,1,27,99,1
1014221,339397889.0,339397889,11.063568,116000620P000034,2020-06-02,2020-06-02,33,3373,9,2,...,Z99,Z999,Dependence on unspecified enabling machine and...,98,9998,98,98,98,98,1
1014222,76213152.0,76213152,286.792694,160421019P000631,2019-10-15,2019-10-15,32,3276,3,1,...,Z99,Z992,Dependence on renal dialysis,32,3276,9,1,33,99,1
1014223,101797983.0,101797983,30.222155,332670919P000083,2019-09-06,2019-09-06,16,1606,3,1,...,Z99,Z992,Dependence on renal dialysis,16,1606,3,1,27,99,1


Example sklearn f1 score
```python
>>> from sklearn.metrics import f1_score
>>> y_true = [0, 1, 2, 0, 1, 2]
>>> y_pred = [0, 2, 1, 0, 0, 1]
>>> f1_score(y_true, y_pred, average='macro')
0.26...
>>> f1_score(y_true, y_pred, average='micro')
0.33...
>>> f1_score(y_true, y_pred, average='weighted')
0.26...
>>> f1_score(y_true, y_pred, average=None)
array([0.8, 0. , 0. ])
>>> y_true = [0, 0, 0, 0, 0, 0]
>>> y_pred = [0, 0, 0, 0, 0, 0]
>>> f1_score(y_true, y_pred, zero_division=1)
1.0...
>>> # multilabel classification
>>> y_true = [[0, 0, 0], [1, 1, 1], [0, 1, 1]]
>>> y_pred = [[0, 0, 0], [1, 1, 1], [1, 1, 0]]
>>> f1_score(y_true, y_pred, average=None)
array([0.66666667, 1.        , 0.66666667])
```

In [11]:
model = LogisticRegression()
model.fit(x_train, y_train)
predicted=model.predict(x_test)
# f1_score(y_test, predicted, average='weighted')

ValueError: could not convert string to float: '449590620P000054'

In [ ]:
model = GaussianNB()
model.fit(x_train, y_train)  
predicted = model.predict(x_test)
f1_score(y_test, predicted, average='weighted')

In [ ]:
model = BernoulliNB()
model.fit(x_train, y_train)
predicted = model.predict(x_test)
f1_score(y_test, predicted, average='weighted')

In [ ]:
model = SVC()
model.fit(x_train, y_train)
predicted = model.predict(x_test)
f1_score(y_test, predicted, average='weighted')

In [ ]:
model = RandomForestRegressor(n_estimators = 100, random_state = 0)  
model.fit(x_train, y_train)  
predicted = model.predict(x_test)
f1_score(y_test, predicted, average='weighted')

In [ ]:
model = KNeighborsClassifier(n_neighbors = 1)  
model.fit(x_train, y_train)
predicted = model.predict(x_test)
f1_score(y_test, predicted, average='weighted')

In [ ]:
model = xgb.XGBClassifier(use_label_encoder=False)
model.fit(x_train, y_train)
predicted = model.predict(x_test)
f1_score(y_test, predicted, average='weighted')